In [ ]:
!pip install "unstructured[all-docs]"
!pip install libmagic-dev poppler-utils tesseract-ocr

In [52]:
# Import necessary modules
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the secret key
# SECRET_KEY = os.getenv()

# # Now you can use the SECRET_KEY in your code
# print(f'SECRET_KEY:  {SECRET_KEY}')

True

In [6]:
# deps
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [7]:
# laoding PDf local
pdf_path = 'transformers.pdf'

In [8]:
# Processing PDF
if pdf_path:
    loader = UnstructuredPDFLoader(file_path = pdf_path)
    data = loader.load()
else:
    print("Upload a PDF")

In [ ]:
# Viewing Processed pdf
data[0].page_content

## Vector Embeddings

In [58]:
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
# Now you can use the API key to initialize the CohereEmbeddings model
embedding_model = CohereEmbeddings(cohere_api_key = COHERE_API_KEY)

In [5]:
# vecotr embeddings & text spiltter & vecotr store deps
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [44]:
# splitting text from pdf and chunking for proper embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 8000, chunk_overlap = 100)
chunks = text_splitter.split_documents(data)

In [45]:
# adding a vectorDB
vector_db = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings_model,
    collection_name = "rag-pdf"
)

## Retrievel from VectorDb

In [22]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [36]:
from langchain_groq import ChatGroq # load groq deps

GROQ_API_KEY = os.getenv("GROQ")
# loading the mixtral llm from groq
llm = ChatGroq(temperature=0, groq_api_key='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', model_name="mixtral-8x7b-32768")

In [37]:
# creating a custom template saying whenever user input a Query, make 5 alternative similar query
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [46]:
# Storing those alternative generated query into vecotor db, this is a custom retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
if not found generate generic answer also along with it mention "NOT FROM PDF"
"""

prompt = ChatPromptTemplate.from_template(template)

In [47]:
# Now lets chain everything
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [48]:
chain.invoke(input(""))

 Derieve 5 questions from the pdf in points


'1. What is the Transformer model and how does it differ from traditional sequence transduction models?\nAnswer: The Transformer model is a type of sequence transduction model that is based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. This makes it more parallelizable and allows it to significantly reduce training time compared to traditional models.\n\n2. How does the Transformer model perform on machine translation tasks?\nAnswer: The Transformer model has been shown to be superior in quality on two machine translation tasks, achieving a BLEU score of 28.4 on the WMT 2014 English-to-German translation task and a BLEU score of 41.8 on the WMT 2014 English-to-French translation task.\n\n3. How does the Transformer model handle long-distance dependencies in sequences?\nAnswer: The Transformer model handles long-distance dependencies in sequences using self-attention mechanisms, which allow it to attend to all positions in the sequence simultaneou